# 📓 The GenAI Revolution Cookbook

**Title:** How to Build Multi-Agent AI Systems with CrewAI, Step by Step

**Description:** Design scalable multi-agent AI systems with CrewAI using YAML configs, task delegation, tools, and training—then build a Customer Feedback Analyst.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



Here’s a revised version of your draft content. I’ve aligned the code and config structure much more closely with *“Building Multi\-Agent AI Systems: A Practical Guide with CrewAI”* from The GenAI Revolution, while preserving your existing narrative, humanizing for clearer flow, and obeying your instruction rules.

---

Generative AI gets useful when it ships. In this guide you'll build a Customer Feedback Analyst using CrewAI. You’ll define agents in YAML, wire tasks and tools, run the system end\-to\-end, and produce a real report with sentiment tables and a pie chart. You’ll learn the exact architecture and code so you can reuse it with your own data. If you like extracting structured data with LLMs, our [structured data extraction pipeline guide](/article/structured-data-extraction-with-llms-how-to-build-a-pipeline-3) walks you through best practices.

## Why This Approach Works

Multi\-agent systems shine when you need specialized roles working together. A single LLM call can classify sentiment, but it struggles to read files, compute aggregates, generate charts, and assemble reports all in one go. By splitting those responsibilities across focused agents, you get modularity, clarity, and you can swap models or tools per task.

CrewAI gives you a clean model for agents, tasks, tools, and crews. You define roles and workflows in YAML. You reduce boilerplate. You run sequential, hierarchical, or parallel processes with built\-in context passing, delegation, and memory. To help your agents interoperate and audit more effectively, take a look at the [Model Context Protocol (MCP)](/article/model-context-protocol-mcp-explained-2025-guide-for-builders).

In this tutorial you’ll use a sequential pipeline. Each task receives context from previous steps. That keeps logic simple and traceable.

## How It Works

The pipeline has five stages:

1. **Read feedback**: A file reader tool loads the CSV.
2. **Classify sentiment**: An agent labels each row as positive, neutral, or negative, outputting JSONL.
3. **Aggregate counts**: A Python tool parses the JSONL and computes sentiment totals deterministically.
4. **Generate chart**: A tool creates a pie chart PNG from the aggregated counts.
5. **Assemble report**: A writer agent combines the table and chart into a Markdown document.

Each agent has a role, goal, backstory, and assigned tools. Tasks declare dependencies via context. The crew orchestrates execution. You’ll get a final report with real artifacts.

## What You’ll Build

By the end, you’ll have:

* A `customer_feedback_report.md` with sentiment breakdown and insights.
* A `sentiment_pie.png` chart visualizing the distribution.
* A reusable pipeline you can adapt to other CSV datasets or feedback sources.

You’ll run everything in a notebook or script. You’ll validate outputs. You’ll understand the design decisions behind each component.

## Setup \& Installation

Start by installing dependencies and pinning versions to stay stable.

In [ ]:
!pip install crewai[tools]==0.28.0 python-dotenv==1.0.0 pyyaml==6.0.1

Set your API keys. If you’re in a notebook without a `.env` file, set them in session:

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

<p>_ = load_dotenv(find_dotenv())</p>
<p>os.environ["OPENAI_API_KEY"] = "your-openai-key-here"
os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-key-here"  # optional, if using Claude

Check the keys:

In [ ]:
required_keys = ["OPENAI_API_KEY"]
missing = [k for k in required_keys if not os.getenv(k)]
if missing:
    raise EnvironmentError(
        f"Missing required environment variables: {', '.join(missing)}"
        " Please set them before running the notebook."
    )
print("All required API keys found.")

## Step\-by\-Step Implementation

### Define Tools

Tools are functions that agents call to carry out actions. You’ll create a file reader, a sentiment aggregator, and a chart generator.

In [ ]:
# tools/__init__.py
from crewai_tools import FileReadTool, tool
import json
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd

<p>def make_file_reader(csv_path: str):
    return FileReadTool(file_path=csv_path, description="Read customer_feedback.csv")</p>
<p>@tool("aggregate_sentiment")
def aggregate_sentiment(jsonl_str: str) -> str:
    lines = [ln.strip() for ln in jsonl_str.strip().split("\n") if ln.strip()]
    records = [json.loads(ln) for ln in lines]
    df = pd.DataFrame(records)
    counts = df["sentiment"].value_counts().to_dict()
    result = {
        "positive": counts.get("positive", 0),
        "neutral": counts.get("neutral", 0),
        "negative": counts.get("negative", 0),
    }
    return json.dumps(result)</p>
<p>@tool("make_sentiment_pie")
def make_sentiment_pie(summary_json: str) -> str:
    data = json.loads(summary_json)
    counts = [data.get("positive", 0), data.get("neutral", 0), data.get("negative", 0)]
    labels = ["Positive", "Neutral", "Negative"]
    colors = ["#2ecc71", "#95a5a6", "#e74c3c"]</p>
<pre><code>reports_dir = Path("reports")
reports_dir.mkdir(parents=True, exist_ok=True)
out_path = reports_dir / "sentiment_pie.png"

fig, ax = plt.subplots(figsize=(5, 5), dpi=160)
ax.pie(counts, labels=labels, autopct="%1.1f%%", startangle=140, colors=colors)
ax.axis("equal")
plt.title("Sentiment Distribution")
plt.tight_layout()
fig.savefig(out_path)
plt.close(fig)
return str(out_path.resolve())</code></pre>

### Configure Agents via YAML

Agents are defined in YAML. Each has a role, goal, backstory, and tools.

In [ ]:
# configs/agents.yaml
sentiment_classifier:
  role: Sentiment Classifier
  goal: Label each feedback entry as positive, neutral, or negative
  backstory: You are an expert in customer tone and sentiment. You read feedback text and assign accurate sentiment labels.
  llm: gpt-4o-mini
  tools:
    - file_reader
  verbose: true
  allow_delegation: false

<p>aggregator:
  role: Data Aggregator
  goal: Compute sentiment counts from classified feedback
  backstory: You process structured data and produce accurate numeric summaries.
  llm: gpt-4o-mini
  tools:
    - aggregate_sentiment
  verbose: true
  allow_delegation: false</p>
<p>chart_maker:
  role: Chart Generator
  goal: Create a pie chart image showing sentiment distribution
  backstory: You turn numbers into professional visuals.
  llm: gpt-4o-mini
  tools:
    - make_sentiment_pie
  verbose: true
  allow_delegation: false</p>
<p>report_writer:
  role: Report Writer
  goal: Assemble a Markdown report with table and chart
  backstory: You synthesize insights and visuals into clear reports for stakeholders.
  llm: gpt-4o-mini
  tools: []
  verbose: true
  allow_delegation: false

### Configure Tasks via YAML

Tasks define what each agent does. You wire outputs between them using \`context\` to enforce order.

In [ ]:
# configs/tasks.yaml
classify_feedback:
  description: |
    Read the customer_feedback.csv file using the file_reader tool.
    For each row, classify sentiment of the feedback_text field as Positive, Neutral, or Negative.
    Output JSONL with id, feedback_text, sentiment per feedback entry.
  agent: sentiment_classifier
  expected_output: JSONL with id, feedback_text, and sentiment

<p>aggregate_counts:
  description: |
    Take the JSONL output from classify_feedback.
    Use the aggregate_sentiment tool to compute total counts for positive, neutral, and negative.
    Return a JSON object with keys: positive, neutral, negative.
  agent: aggregator
  expected_output: JSON object with sentiment counts.
  context:
    - classify_feedback</p>
<p>generate_chart:
  description: |
    Take the summary JSON from aggregate_counts.
    Use make_sentiment_pie tool to generate a pie chart PNG.
    Return the absolute file path to the generated chart image.
  agent: chart_maker
  expected_output: Absolute path to sentiment_pie.png.
  context:
    - aggregate_counts</p>
<p>write_report:
  description: |
    Using sentiment counts (from aggregate_counts) and chart path (from generate_chart),
    write a Markdown report that includes:
    - Summary of total feedback entries and sentiment breakdown.
    - Table of positive, neutral, negative counts.
    - Embedded image: <img src="reports/sentiment_pie.png" alt="Sentiment Distribution">
    - Brief insights and recommendations based on sentiment distribution.
    Return the complete Markdown content.
  agent: report_writer
  expected_output: Complete Markdown report with table, chart, and insights.
  context:
    - aggregate_counts
    - generate_chart

### Instantiate Agents, Tasks \& Crew

In [ ]:
import os
from pathlib import Path
import yaml
from dotenv import load_dotenv, find_dotenv
from crewai import Agent, Task, Crew
from tools import make_file_reader, aggregate_sentiment, make_sentiment_pie

_ = load_dotenv(find_dotenv())

# Paths
ROOT = Path.cwd()
DATA_DIR = ROOT / "data"
CONFIGS_DIR = ROOT / "configs"
REPORTS_DIR = ROOT / "reports"

csv_path = DATA_DIR / "customer_feedback.csv"

# Build tools
tools_registry = {
    "file_reader": make_file_reader(str(csv_path)),
    "aggregate_sentiment": aggregate_sentiment,
    "make_sentiment_pie": make_sentiment_pie,
}

# Load YAML configs
def load_yaml(path: Path):
    if not path.exists():
        raise FileNotFoundError(f"Missing config: {path}")
    import yaml
    return yaml.safe_load(path.read_text())

agents_cfg = load_yaml(CONFIGS_DIR / "agents.yaml")
tasks_cfg = load_yaml(CONFIGS_DIR / "tasks.yaml")

# Instantiate agents
agents = {}
for name, spec in agents_cfg.items():
    agent = Agent(
        role=spec["role"],
        goal=spec["goal"],
        backstory=spec.get("backstory", ""),
        llm=spec.get("llm"),
        tools=[tools_registry[t] for t in spec.get("tools", [])],
        verbose=spec.get("verbose", False),
        allow_delegation=spec.get("allow_delegation", False),
    )
    agents[name] = agent

# Instantiate tasks
tasks = {}
for name, spec in tasks_cfg.items():
    task = Task(
        description=spec["description"],
        expected_output=spec["expected_output"],
        agent=agents[spec["agent"]],
    )
    # assign context if provided
    if "context" in spec:
        task.context = [tasks[c] for c in spec["context"]]
    tasks[name] = task

# Assemble Crew
crew = Crew(
    agents=list(agents.values()),
    tasks=list(tasks.values()),
    process=Crew.Process.sequential,
    verbose=True,
)

print("Starting pipeline...")
final_report = crew.kickoff()

# Save outputs
REPORTS_DIR.mkdir(parents=True, exist_ok=True)
(report_path := REPORTS_DIR / "customer_feedback_report.md").write_text(final_report, encoding="utf-8")
print(f"Report saved at {report_path.resolve()}")

chart_path = REPORTS_DIR / "sentiment_pie.png"
if chart_path.exists():
    print(f"Chart saved at {chart_path.resolve()}")
else:
    print("Warning: Chart not found. Check tool execution logs.")

## Key Design Decisions

**Deterministic aggregation:** Asking an LLM to compute counts risks hallucination or errors. The Python tool using pandas produces accurate totals every time.

**Sequential execution:** You depend on prior outputs: classify before aggregating, chart before reporting. Sequential tasks keep the process simple and easy to debug.

**YAML for configuration:** Defining agents and tasks in YAML makes the system maintainable. You can swap models, adjust prompts, or add agents without touching orchestration code.

**Single\-responsibility tools:** Each tool does one thing. That lets you test, reuse, or replace parts independently.

## Conclusion

You’ve built a complete Customer Feedback Analyst using CrewAI. You defined agents in YAML. You wired tasks with clear context dependencies. You attached tools for reading files, aggregating data, and charting. You ran the system end to end to produce a Markdown report and PNG visualization.

This architecture is reusable. Swap in a different CSV. Adjust the classifier prompt. Add agents for deeper analysis. You now have a template for multi\-agent pipelines that coordinate specialized roles to deliver structured, validated outputs.

**Next steps:**

* Add schema validation for JSONL outputs using Pydantic. That helps catch malformed data early.
* Implement chunking for large CSVs to avoid context length limits.
* Add observability: log token usage, task timings, and tool calls for performance and cost tracking.
* Explore parallel task execution for independent steps to reduce latency.
* Consider deploying the pipeline as an API or scheduled job for continuous feedback analysis.

---

### References

* Building Multi\-Agent AI Systems: A Practical Guide with CrewAI. The GenAI Revolution. ([thegenairevolution.com](https://thegenairevolution.com/building-multi-agent-ai-systems-a-practical-guide-with-crewai/))